# Strategie Pattern

Das Strategy Pattern ist ein Entwurfsmuster, das es unserer Anwendung ermöglicht, Algorithmen zur Laufzeit auszuwählen, was unsere Anwendung flexibel macht. Im Originalbuch über Entwurfsmuster von GoF heißt es: "Das Strategy Pattern zielt darauf ab, eine Familie von Algorithmen zu definieren, jeden einzelnen zu kapseln und sie austauschbar zu machen." Genauer gesagt, ermöglicht es Ihnen, eine Reihe von Algorithmen zu definieren, die während der Laufzeit in Abhängigkeit von einigen Faktoren austauschbar sind. Das Strategy Pattern fällt unter die Kategorie der Verhaltensmuster (Behavioral Design Patterns), da es die Auswahl des Verhaltens eines Algorithmus zur Laufzeit ermöglicht.

Wenn du Softwareanwendungen entwickelst, hast du vielleicht ein paar Alternativen, um etwas in deinem Code zu erreichen. Je nach Kundenwahl, Datenquellen oder anderen Faktoren möchtest du etwas anders machen, ohne den Code zu ändern. Oft neigst du dazu, Algorithmen mit bedingten Anweisungen für verschiedene Situationen in der Hauptklasse des Codes zu definieren. Aber das ist kein eleganter Weg, um besseren Code zu schreiben. Die Hauptklasse deines Codes wird dadurch ziemlich lang, und es wird zu schwierig, die Anwendung zu warten.

In solchen Situationen ist das Strategy Pattern die ideale Lösung. Das Strategy Pattern schlägt vor, dass du für unterschiedliche Strategien, Klassen definierst, die du für deine Algorithmen verwendest. Die Strategie wird in der Hauptklasse referenziert, die als Kontext bezeichnet wird, und der Code arbeitet entsprechend der jeweiligen Situation. Der Kontext wählt keine geeignete Strategie für den jeweiligen Fall aus. Stattdessen übergibt der Kunde die gewünschte Strategie an den Kontext.

Das Strategy Pattern folgt dem Open/Close-Prinzip: Eine Softwareanwendung ist offen für Erweiterungen, aber geschlossen für Veränderungen. Das bedeutet, dass du eine beliebige Anzahl von zusätzlichen Strategien hinzufügen kannst, ohne die Hauptklasse zu verändern. Das macht deinen Code flexibler und leichter zu pflegen.

#### Ohne Strategie Pattern

In [82]:
import random
import string
from typing import List

class SupportTicket:
    id: str
    customer: str
    issue: str
    
    def __init__(self, customer, issue) -> None:
        self.id = self._generate_id(10)
        self.customer = customer
        self.issue = issue
        
    def _generate_id(self, length: int) -> str:
        return ''.join(random.choices(string.ascii_letters + string.digits, k=length))
    

class CustomerSupport:
    
    tickets: List[SupportTicket] = []
    
    def add_ticket(self, support_ticket) -> None:
        self.tickets.append(support_ticket)
        
    def process_tickets(self, process_strategie: str = "FIFO") -> None:
        
        if not len(self.tickets):
            print("Es sind zur Zeit keine Support Tickets vorhanden!")
            return
        
        if process_strategie.lower() == "fifo":
            for ticket in self.tickets:
                self._process_ticket(ticket)
        elif process_strategie.lower() == "filo":
            for ticket in reversed(self.tickets):
                self._process_ticket(ticket)
        elif process_strategie.lower() == "random":
            tickets_copy = self.tickets.copy()
            random.shuffle(self.tickets)
            for ticket in tickets_copy:
                self._process_ticket(ticket)
        else:
            print(f"Die Process Strategie '{process_strategie}' ist nicht vorhanden!")
                
    @staticmethod
    def _process_ticket(ticket: SupportTicket) -> None:
        print("==================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("==================================")
            
            
customer_support = CustomerSupport()
customer_support.add_ticket(SupportTicket("Davie Johnes", "Meine Maus ruckelt!"))
customer_support.add_ticket(SupportTicket("Marie Antoinette", "Git Plugin für VScode funktioniert nicht"))
customer_support.add_ticket(SupportTicket("Jordan Peters", "Habe ein Problem"))

customer_support.process_tickets('random')

Processing ticket id: 0tzjICRMwB
Customer: Davie Johnes
Issue: Meine Maus ruckelt!
Processing ticket id: dwC61S5YpF
Customer: Marie Antoinette
Issue: Git Plugin für VScode funktioniert nicht
Processing ticket id: PVAiwnNkjr
Customer: Jordan Peters
Issue: Habe ein Problem


#### Mit Strategie Pattern

In [83]:
import random
import string
from typing import List
from abc import ABC, abstractmethod


class TicketOrderStrategie(ABC):

    @abstractmethod
    def create_ordering(self, list: List[SupportTicket]) -> List[SupportTicket]:
        pass


class FIFOrderStrategie(TicketOrderStrategie):

    def create_ordering(self, list: List[SupportTicket]) -> List[SupportTicket]:
        return list.copy()


class RandomOrderStrategie(TicketOrderStrategie):

    def create_ordering(self, list: List[SupportTicket]) -> List[SupportTicket]:
        list_copy = list.copy()
        random.shuffle(list_copy)
        return list_copy


class FILOOrderStrategie(TicketOrderStrategie):

    def create_ordering(self, list: List[SupportTicket]) -> List[SupportTicket]:
        list_copy = list.copy()
        list_copy.reverse()
        return list_copy


class CustomerSupport2:

    tickets: List[SupportTicket] = []

    def add_ticket(self, support_ticket) -> None:
        self.tickets.append(support_ticket)

    def process_tickets(self, process_strategie: TicketOrderStrategie) -> None:
        ticket_list = process_strategie.create_ordering(self.tickets)

        if not len(ticket_list):
            print("Es sind zur Zeit keine Support Tickets vorhanden!")
            return

        for ticket in ticket_list:
            self._process_ticket(ticket)

    @staticmethod
    def _process_ticket(ticket: SupportTicket) -> None:
        print("==================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("==================================")


customer_support = CustomerSupport2()
customer_support.add_ticket(SupportTicket("Davie Johnes", "Meine Maus ruckelt!"))
customer_support.add_ticket(SupportTicket("Marie Antoinette", "Git Plugin für VScode funktioniert nicht"))
customer_support.add_ticket(SupportTicket("Jordan Peters", "Habe ein Problem"))

customer_support.process_tickets(RandomOrderStrategie())


Processing ticket id: rSunW8G3Ts
Customer: Davie Johnes
Issue: Meine Maus ruckelt!
Processing ticket id: SxIirEQN5f
Customer: Marie Antoinette
Issue: Git Plugin für VScode funktioniert nicht
Processing ticket id: Nx9XgbE1Jm
Customer: Jordan Peters
Issue: Habe ein Problem


#### Stratagie Pattern mit Protocol

In [93]:
import random
import string
from typing import Protocol


class TicketOrderStrategie(Protocol):
    def create_ordering(self, list: list[SupportTicket]) -> list[SupportTicket]: 
        """Gibt eine geordnete Liste mit Tickets zurück"""


class FIFOrderStrategie:
    def create_ordering(self, list: list[SupportTicket]) -> list[SupportTicket]:
        return list.copy()


class RandomOrderStrategie:
    def create_ordering(self, list: list[SupportTicket]) -> list[SupportTicket]:
        list_copy = list.copy()
        random.shuffle(list_copy)
        return list_copy


class FILOOrderStrategie:
    def create_ordering(self, list: list[SupportTicket]) -> list[SupportTicket]:
        list_copy = list.copy()
        list_copy.reverse()
        return list_copy


class CustomerSupport2:

    tickets: List[SupportTicket] = []

    def add_ticket(self, support_ticket) -> None:
        self.tickets.append(support_ticket)

    def process_tickets(self, process_strategie: TicketOrderStrategie) -> None:
        ticket_list = process_strategie.create_ordering(self.tickets)

        if not len(ticket_list):
            print("Es sind zur Zeit keine Support Tickets vorhanden!")
            return

        for ticket in ticket_list:
            self._process_ticket(ticket)

    @staticmethod
    def _process_ticket(ticket: SupportTicket) -> None:
        print("==================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("==================================")


customer_support = CustomerSupport2()
customer_support.add_ticket(SupportTicket("Davie Johnes", "Meine Maus ruckelt!"))
customer_support.add_ticket(SupportTicket("Marie Antoinette", "Git Plugin für VScode funktioniert nicht"))
customer_support.add_ticket(SupportTicket("Jordan Peters", "Habe ein Problem"))

customer_support.process_tickets(RandomOrderStrategie())


Processing ticket id: FrX4wklPYy
Customer: Jordan Peters
Issue: Habe ein Problem
Processing ticket id: Fnz06l3kWj
Customer: Davie Johnes
Issue: Meine Maus ruckelt!
Processing ticket id: FTerVxW8em
Customer: Marie Antoinette
Issue: Git Plugin für VScode funktioniert nicht


#### Dunder Methode (\_\_call\_\_)

In [95]:
import random
import string
from typing import Protocol


class TicketOrderStrategie(Protocol):
    def __call__(self, list: list[SupportTicket]) -> list[SupportTicket]: 
        """Gibt eine geordnete Liste mit Tickets zurück"""


class FIFOrderStrategie:
    def __call__(self, list: list[SupportTicket]) -> list[SupportTicket]:
        return list.copy()


class RandomOrderStrategie:
    def __call__(self, list: list[SupportTicket]) -> list[SupportTicket]:
        list_copy = list.copy()
        random.shuffle(list_copy)
        return list_copy


class FILOOrderStrategie:
    def __call__(self, list: list[SupportTicket]) -> list[SupportTicket]:
        list_copy = list.copy()
        list_copy.reverse()
        return list_copy


class CustomerSupport2:

    tickets: List[SupportTicket] = []

    def add_ticket(self, support_ticket) -> None:
        self.tickets.append(support_ticket)

    def process_tickets(self, process_strategie: TicketOrderStrategie) -> None:
        ticket_list = process_strategie(self.tickets)

        if not len(ticket_list):
            print("Es sind zur Zeit keine Support Tickets vorhanden!")
            return

        for ticket in ticket_list:
            self._process_ticket(ticket)

    @staticmethod
    def _process_ticket(ticket: SupportTicket) -> None:
        print("==================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("==================================")


customer_support = CustomerSupport2()
customer_support.add_ticket(SupportTicket("Davie Johnes", "Meine Maus ruckelt!"))
customer_support.add_ticket(SupportTicket("Marie Antoinette", "Git Plugin für VScode funktioniert nicht"))
customer_support.add_ticket(SupportTicket("Jordan Peters", "Habe ein Problem"))

customer_support.process_tickets(RandomOrderStrategie())


Processing ticket id: wmcjqiLRnb
Customer: Davie Johnes
Issue: Meine Maus ruckelt!
Processing ticket id: 7vCnv4FMNm
Customer: Jordan Peters
Issue: Habe ein Problem
Processing ticket id: bUdiH3lgq1
Customer: Marie Antoinette
Issue: Git Plugin für VScode funktioniert nicht


#### Einfache Funktionale Methode

In [92]:
import random
import string
from typing import List, Callable

def fifo_ordering(list: List[SupportTicket]) -> List[SupportTicket]:
    return list.copy()

def random_ordering(list: List[SupportTicket]) -> List[SupportTicket]:
    list_copy = list.copy()
    random.shuffle(list_copy)
    return list_copy

def filo_ordering(list: List[SupportTicket]) -> List[SupportTicket]:
    list_copy = list.copy()
    list_copy.reverse()
    return list_copy


class CustomerSupport3:

    tickets: List[SupportTicket] = []

    def add_ticket(self, support_ticket) -> None:
        self.tickets.append(support_ticket)

    def process_tickets(self, process_strategie_function: Callable[[List[SupportTicket]], List[SupportTicket]]) -> None:
        ticket_list = process_strategie_function(self.tickets)

        if not len(ticket_list):
            print("Es sind zur Zeit keine Support Tickets vorhanden!")
            return

        for ticket in ticket_list:
            self._process_ticket(ticket)

    @staticmethod
    def _process_ticket(ticket: SupportTicket) -> None:
        print("==================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("==================================")


customer_support = CustomerSupport3()
customer_support.add_ticket(SupportTicket("Davie Johnes", "Meine Maus ruckelt!"))
customer_support.add_ticket(SupportTicket("Marie Antoinette", "Git Plugin für VScode funktioniert nicht"))
customer_support.add_ticket(SupportTicket("Jordan Peters", "Habe ein Problem"))

customer_support.process_tickets(random_ordering)


Processing ticket id: 7UOjHdTUuK
Customer: Davie Johnes
Issue: Meine Maus ruckelt!
Processing ticket id: iBUpW3BZBG
Customer: Jordan Peters
Issue: Habe ein Problem
Processing ticket id: KnJ2FniEPB
Customer: Marie Antoinette
Issue: Git Plugin für VScode funktioniert nicht
